## Семинар 8: "Современные модели для NLP"

ФИО: Вахрушев Вадим Юрьевич

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [1]:
import torch
!pip install transformers
from transformers import *

In [2]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [3]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [4]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [5]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [6]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
print(input_batch)
with torch.no_grad():
    res = model(input_batch)[0]

tensor([[  101,  3446,  1110,  1199,   103,  1106,  4035, 13775,   102]])


In [7]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [8]:
new_ids

tensor([[  119,  1303,  1110,  1199,  1236,  1106,  4035, 13775,   119]])

In [9]:
tokenizer.decode(new_ids.numpy()[0].tolist())

'. here is some way to encode.'

In [10]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [11]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)

In [12]:
input_ids

[101,
 1130,
 170,
 19196,
 4006,
 117,
 7482,
 2751,
 170,
 17804,
 1104,
 8362,
 23941,
 1116,
 1690,
 1107,
 170,
 6456,
 117,
 2331,
 25731,
 1775,
 1643,
 21425,
 1181,
 4524,
 117,
 1107,
 1103,
 19505,
 5249,
 119,
 2431,
 1167,
 11567,
 1106,
 1103,
 6962,
 1108,
 1103,
 1864,
 1115,
 1103,
 8362,
 23941,
 1116,
 2910,
 3264,
 1483,
 119,
 102]

In [14]:
from tqdm import tqdm

In [15]:
for i in tqdm(range(1000)):
    input_ids.insert(len(input_ids) - 1,tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    predicted_index = prob.max(-1)[1][0].numpy()
    input_ids[len(input_ids) -2] = predicted_index[len(input_ids[i:]) - 2]

100%|██████████| 1000/1000 [03:15<00:00,  5.10it/s]


In [16]:
tokenizer.decode(input_ids)

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. ” ” ” ”. ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ”

получилось не очень хорошо, попробуем семплировать

In [19]:


input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)

for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    sampler = torch.distributions.Categorical(prob)
    predicted_index = sampler.sample().data.numpy()
    input_ids[len(input_ids) - 2] = predicted_index[0][len(input_ids[i:]) - 2]
    



In [21]:
tokenizer.decode(input_ids)

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. org. uk. com.?.? > > <. > > > > | > > <? > > < / > > But!!!!! ¡ > > " <! > > "\'>? > | > >?. > > > > >? > > " ॥ | | | "don > | | ] | | } | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | ॥ | | | | | | | | | | | | | | | | | | | | | | | | | | | | | ॥ | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |

тоже не очень, попробуем брать случайные слова из наиболее вероятных

In [11]:
from random import randint

In [13]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)

for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    predicted_index = torch.topk(prob, 100)[1][0][:,randint(0, 99)].numpy()
    input_ids[len(input_ids) - 2] = predicted_index[len(input_ids[i:]) - 2]
    
tokenizer.decode(input_ids)

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The information garnered incredible visibility overnight so vividl about supernatural beings while experiencing transformation rapidly during excavation period... 33 days { this " ’ is that —. < xiki website explanation gallery timeline display history edition viewr website source pages database search icon photo search indicator results listing section 1! %? http links from infoframe services section § 4636 § 141 ~ " infofront ″ preview - preview |? ） 2014 г ⟩я article editr archive interface update presentation ) 1993 cln 2005036 » report # 55 * 13922 )... 2009117064 } _ : 55272041158600 ″ 5 index16 # 16 | 5 ） reference manual... ] * editor `, 1pm 1973 + 8020°807ʰ ] > editory " ′ article 64861937° `f161927\'36 ′ʰ, [UNK] coordinates & notes ’ § 365518

уже что-то, отдаленно напрминающее текст. Проделаем аналогичные операции с другим текстом

In [28]:
input_ids = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)

for i in tqdm(range(1000)):
    input_ids.insert(len(input_ids) - 1,tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    predicted_index = prob.max(-1)[1][0].numpy()
    input_ids[len(input_ids) -2] = predicted_index[len(input_ids[i:]) - 2]

100%|██████████| 1000/1000 [02:12<00:00,  7.54it/s]


In [29]:
tokenizer.decode(input_ids)

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. " ". " ". " ". " ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". 

In [30]:
input_ids = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)
for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    sampler = torch.distributions.Categorical(prob)
    predicted_index = sampler.sample().data.numpy()
    input_ids[len(input_ids) - 2] = predicted_index[0][len(input_ids[i:]) - 2]

In [32]:
tokenizer.decode(input_ids)

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. ” statistics include : |... | | } | } | practice | { } | | | } { } | } } } { } } { } } ; } } } } { } } } { } } } } { { } } { } } \\ { } } } { } { } } } { } } } { } { { } } { } } { } } } } { } } { } } } } { } } ^ { } } { { } } { } } } \\ } { } } } ^ { } } } { \\ }! } } { } } } { } } { } } { } } } } } { } } { } } } { } } { { } } } } \\ { } { } } } } { { } } } { } } { } } { } } { } } } } } { } } } { | | } } } { } } { } } } } { } } { } } { } } } { } } } { { } } { } { } } { } } { } } } { } } } { { } } { } } { } } { } } { } } } { } } { } } { } } } { } } { } { } } } { } { } ^ { } } } } { } } } { } } { } } } { } } { } } { { } } { } } } } } { } } { } } } { } } { } } } { } } } { } } { } | } { } } { } } } { } } { } } { } } } { } } } { } } } { } } { } } { } { } } { { } } } { { } } { } } { } } } ; } { { } } } { { } } } } { } } { { } } } { } } } } } { } } { } } } { } } { } } }

In [33]:
input_ids = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)

for i in range(1000):
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0) # batch_size = 1
    with torch.no_grad():
        predictions = model(input_batch)[0]
    prob = torch.nn.functional.softmax(predictions, dim=-1)
    predicted_index = torch.topk(prob, 100)[1][0][:,randint(0, 99)].numpy()
    input_ids[len(input_ids) - 2] = predicted_index[len(input_ids[i:]) - 2]
    
tokenizer.decode(input_ids)

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. ; - 6 squadron! 377th class Infantry reserve vessel operator officer 7 operations sergeant sergeant pilot 74º troop eight destroyer crew compartment 28 ′ 17 ″ "! station 5259045 } address 35312843 } ^ page 35 ″ > row 5341775 ) * website commentator { node2148404r... 8 ॥ 8 ¶ 28 ′ 9dᵐ... ; 83ᵉ 36 )¹2826¹ · 2821⁰23 \\ ombre coordinates ： 1 ] : • ‡4635 ; | 5 _ x5x } 9am, > 1430407533! \\ 5qm15? { 64748r2A1234 । { 16, 192 ) ] † 2ᵉ2621479 » # # 77193211³719 } = 7 ।°402º53194311² _ 311035747869 / 05. 0864470¹901539 @ { linkshott key162526 # template message + document | | tabled element + html boxcode function » » } type 3 } function name, ⟩ template\'name of entity } in ) or? define database { return. rowan # array : relation ifrtifoof program void statement space overlines satisfying elements variables function composition element generators element satisfy } | and de

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: